## Create a calibration using pyFAI

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pyFAI.gui.jupyter
import pyFAI
import fabio
from pyFAI.test.utilstest import UtilsTest
from pyFAI.calibrant import CALIBRANT_FACTORY
from pyFAI.goniometer import SingleGeometry
import pathlib
from tqdm import tqdm
import logging
import math
import warnings
import os
import pyFAI.azimuthalIntegrator
from pyFAI.gui import jupyter
import yaml

import pyFAI_analysis_functions as analysis
%load_ext autoreload
%autoreload 2

from typing import Tuple
from typing import List

figure_size = 10

## Load YAML file

The file paths and user inputs for the analysis of the Diamond 2017, Diamond 2021 and DESY 2021 experiments are included in the `yaml` configuration files, to record the inputs of the analysis.

The information in the `yaml` file can be accessed like this.

In [ ]:
# to load the Diamond 2017 analysis
config_path = "yaml/config_diamond_2017.yaml"
config = analysis.get_config(config_path)

In [ ]:
# to load the Diamond 2021 analysis at ~740mm
config_path = "yaml/config_diamond_2021_740mm.yaml"
config = analysis.get_config(config_path)

In [ ]:
# to load the Diamond 2021 analysis at ~910mm
config_path = "yaml/config_diamond_2021_910mm.yaml"
config = analysis.get_config(config_path)

In [ ]:
# to load the DESY 2021 analysis
config_path = "yaml/config_desy_2021.yaml"
config = analysis.get_config(config_path)

## Calibrate the detector

Define geometry components such as beam centre and detector distance.

In [ ]:
input_calibrant_file = config["file_paths"]["input_calibrant_file"]
print("The input file path for the calibrant diffraction pattern image is:", input_calibrant_file)
output_calibration_path = config["file_paths"]["output_calibration_path"]
print("The output file path for the .poni calibration file is: ", output_calibration_path)

beam_centre_x = config["setup_inputs"]["beam_centre_x"]
print("The x-coordinate of the beam-centre in pixels is ", beam_centre_x)
beam_centre_y = config["setup_inputs"]["beam_centre_y"] 
print("The y-coordinate of the beam-centre in pixels is ", beam_centre_y)

sample_detector_distance = config["setup_inputs"]["sample_detector_distance"] 
print("The sample-to-detector distance in millimetres (unit used by Fit2d) is: ", sample_detector_distance)

wl = config["setup_inputs"]["wl"] 
print("The beam wavelength in metres is: ", wl)

pixel_x = config["setup_inputs"]["pixel_x"] 
print("The pixel size in metres (in x) is: ", pixel_x)
pixel_y = config["setup_inputs"]["pixel_y"]
print("The pixel size in metres (in y) is: ", pixel_y) 

calibrant_type = config["setup_inputs"]["calibrant_type"] 
print("The calibrant type is: ", calibrant_type)

num_calibration_rings = config["setup_inputs"]["num_calibration_rings"]
print("The number of calibrant rings used for calibration is: ", num_calibration_rings)

In [ ]:
analysis.calibrate(input_calibrant_file, beam_centre_x, beam_centre_y, sample_detector_distance, wl, pixel_x, pixel_y, 
          calibrant_type, num_calibration_rings, output_calibration_path, figure_size)

# Load calibration

In [ ]:
ai = pyFAI.load(output_calibration_path)
print("\nIntegrator: \n", ai)

## Create mask

In [ ]:
input_calibrant_file = config["file_paths"]["input_calibrant_file"]
print("The input file path for the calibrant diffraction pattern image is:", input_calibrant_file)

image = fabio.open(input_calibrant_file)
pattern_image_array = image.data
print("pattern_image_array:", type(pattern_image_array), pattern_image_array.shape, pattern_image_array.dtype)
jupyter.display(pattern_image_array)

mask = pattern_image_array < 0
print("mask:", type(mask), mask.shape, mask.dtype)
print("percent of bad pixels : ", 100*mask.sum()/(mask.shape[0]*mask.shape[1]))
jupyter.display(mask)

# Azimuthal integration of multiple images for `TOPAS`

In [ ]:
input_path = config["file_paths"]["input_path"]
print("The input path is: ", input_path)

input_experiment_list = config["file_paths"]["input_experiment_list"]
print("The input experiment list (name of the input folders) is :", input_experiment_list)

glob_search_term = config["file_paths"]["glob_search_term"]
print("The glob search term (common name of the image files) is :", glob_search_term)

output_path = config["file_paths"]["output_path"]
print("The output path is: ", output_path)

output_experiment_list = config["file_paths"]["output_experiment_list"]
print("The output experiment list (name of the output folders) is :", input_experiment_list)

number_of_points = config["refinement_inputs"]["number_of_points"]
print("The number of points to resolve the intensity profile is :", number_of_points)

In [ ]:
analysis.azimuthal_integration_iteration(input_path, input_experiment_list, glob_search_term, output_path, 
                                         output_experiment_list, ai, mask, number_of_points)

# Caking multiple images for `xrdfit`

In [ ]:
input_path = config["file_paths"]["input_path"]
print("The input path is: ", input_path)

input_experiment_list = config["file_paths"]["input_experiment_list"]
print("The input experiment list (name of the input folders) is :", input_experiment_list)

glob_search_term = config["file_paths"]["glob_search_term"]
print("The glob search term (common name of the image files) is :", glob_search_term)

output_path = config["file_paths"]["output_path"]
print("The output path is: ", output_path)

output_experiment_list = config["file_paths"]["output_experiment_list"]
print("The output experiment list (name of the output folders) is :", input_experiment_list)

number_of_points = config["refinement_inputs"]["number_of_points"]
print("The number of points to resolve the intensity profile is :", number_of_points)

number_of_cakes = config["refinement_inputs"]["number_of_cakes"]
print("The number of cakes for 'caking' the synchrotron diffraction pattern images is: ", number_of_cakes, " giving a cake width of ", 360/number_of_cakes, " degrees")

In [ ]:
analysis.caking_iteration_xrdfit(input_path, input_experiment_list, glob_search_term, output_path, output_experiment_list, 
                        ai, mask, number_of_points, number_of_cakes)

## Caking multiple images for `MAUD`

In [ ]:
input_path = config["file_paths"]["input_path"]
print("The input path is: ", input_path)

input_experiment_list = config["file_paths"]["input_experiment_list"]
print("The input experiment list (name of the input folders) is :", input_experiment_list)

glob_search_term = config["file_paths"]["glob_search_term"]
print("The glob search term (common name of the image files) is :", glob_search_term)

output_path = config["file_paths"]["output_path"]
print("The output path is: ", output_path)

output_experiment_list = config["file_paths"]["output_experiment_list"]
print("The output experiment list (name of the output folders) is :", input_experiment_list)

number_of_points = config["refinement_inputs"]["number_of_points"]
print("The number of points to resolve the intensity profile is :", number_of_points)

number_of_cakes = config["refinement_inputs"]["number_of_cakes"]
print("The number of cakes for 'caking' the synchrotron diffraction pattern images is: ", number_of_cakes, " giving a cake width of ", 360/number_of_cakes, " degrees")

pixel_size = config["refinement_inputs"]["pixel_size"]
print("The pixel size in millimetres is: ", pixel_size) 

In [ ]:
analysis.caking_iteration_maud(input_path, input_experiment_list, glob_search_term, output_path, 
                               output_experiment_list, ai, mask, pixel_size, number_of_points, number_of_cakes)